<a href="https://colab.research.google.com/github/ducviet00/Data-Science-DevC-2020/blob/master/Week5_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Oct 12 06:48:18 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset

Let's first of all have a look at the data. You can download the file `labeledTrainData.tsv` on the [Kaggle website of the competition](https://www.kaggle.com/c/word2vec-nlp-tutorial/data), or on our [Google Drive](https://drive.google.com/file/d/1a1Lyn7ihikk3klAX26fgO3YsGdWHWoK5/view?usp=sharing)


In [2]:
!pip install transformers

     |████████████████████████████████| 1.1MB 2.8MB/s 
     |████████████████████████████████| 1.1MB 15.3MB/s 
     |████████████████████████████████| 890kB 19.3MB/s 
     |████████████████████████████████| 3.0MB 33.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=94565f176a1527e9b37e88312ec91709094c043f074112e5f247d7529a471b40
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [99]:
# Import pandas, numpy
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizer, DistilBertModel
import logging
logging.basicConfig(level=logging.ERROR)

In [4]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [5]:
!wget https://github.com/ducviet00/Data-Science-DevC-2020/raw/master/labeledTrainData.tsv.zip
!unzip *.zip

--2020-10-12 06:48:35--  https://github.com/ducviet00/Data-Science-DevC-2020/raw/master/labeledTrainData.tsv.zip
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ducviet00/Data-Science-DevC-2020/master/labeledTrainData.tsv.zip [following]
--2020-10-12 06:48:35--  https://raw.githubusercontent.com/ducviet00/Data-Science-DevC-2020/master/labeledTrainData.tsv.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13585269 (13M) [application/zip]
Saving to: ‘labeledTrainData.tsv.zip’

labeledTrainData.ts 100%[===================>]  12.96M  27.9MB/s    in 0.5s    

2020-10-12 06:48:39 (27.9 MB/s) - ‘labeledTr

In [65]:
# Read dataset with extra params sep='\t', encoding="latin-1"
data = pd.read_csv('labeledTrainData.tsv', sep='\t', encoding='latin-1')

In [66]:
data['review'] = data['review'].str.replace(r'(<[^>]+>)', repl=' ')

In [67]:
data['review_len'] = data['review'].apply(lambda x: len(x.split()))

In [68]:
data

,id,sentiment,review,review_len
0,5814_8,1,With all this stuff going down at the moment w...,429
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...",158
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,377
3,3630_4,0,It must be assumed that those who praised this...,377
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,367
...,...,...,...,...
24995,3453_3,0,It seems like more consideration has gone into...,87
24996,5064_1,0,I don't believe they made this film. Completel...,174
24997,10905_3,0,"Guy is a loser. Can't get girls, needs to buil...",126
24998,10194_3,0,This 30 minute documentary BuÃ±uel made in the...,197


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          25000 non-null  object
 1   sentiment   25000 non-null  int64 
 2   review      25000 non-null  object
 3   review_len  25000 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 781.4+ KB


## 2. Preprocessing

In [100]:
# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 512
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 3
LEARNING_RATE = 1e-05
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', truncation=True, do_lower_case=True)

In [112]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.comment_text = dataframe.review
        self.targets = self.data.sentiment
        self.max_len = max_len

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, index):
        comment_text = str(self.comment_text[index])
        comment_text = " ".join(comment_text.split())

        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [102]:
# Creating the dataset and dataloader for the neural network

train_size = 0.8
train_dataset=data.sample(frac=train_size,random_state=200)
test_dataset=data.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (25000, 4)
TRAIN Dataset: (20000, 4)
TEST Dataset: (5000, 4)


In [103]:
training_set[5]['targets']

tensor(0.)

In [104]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

## 3. Create Model and Train 


In [153]:
# Creating the customized model, by adding a drop out and a dense layer on top of bert to get the final output for the model. 

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bertbase = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, 1)
        self.activation = torch.nn.Sigmoid() 
    
    def forward(self, ids, mask):
        output_1 = self.bertbase(input_ids=ids, attention_mask=mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        output = self.activation(output)
        return torch.squeeze(output)

model = BERTClass()
model.to(device)

BERTClass(
  (bertbase): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_featu

In [154]:
# Just using distilBERT as feature extraction model.
for param in model.bertbase.parameters():
    param.requires_grad = False

In [155]:
loss_fn = torch.nn.BCELoss()

In [156]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [159]:
def train(epoch):
    model.train()
    total_loss = 0
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask)
        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        total_loss += loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    avg_loss = total_loss / len(training_loader)
    print(f'Epoch: {epoch}, Loss:  {avg_loss}')

In [172]:
for epoch in tqdm(range(20)):
    train(epoch)



  0%|          | 0/20 [00:00<?, ?it/s]

  5%|▌         | 1/20 [03:09<1:00:04, 189.73s/it]

Epoch: 0, Loss:  0.3391071856021881




 10%|█         | 2/20 [06:19<56:53, 189.65s/it]  

Epoch: 1, Loss:  0.339933305978775




 15%|█▌        | 3/20 [09:28<53:43, 189.60s/it]

...

Epoch: 17, Loss:  0.327468603849411




 95%|█████████▌| 19/20 [59:59<03:09, 189.41s/it]

Epoch: 18, Loss:  0.3228004276752472




100%|██████████| 20/20 [1:03:09<00:00, 189.46s/it]

Epoch: 19, Loss:  0.32565584778785706


## 4. Evaluate Model

In [161]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [173]:
def validation(testing_loader):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(outputs.cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [174]:
outputs, eval_targets = validation(testing_loader)

final_outputs = np.array(outputs) >=0.5

In [175]:
predictions = final_outputs
print('accuracy:',accuracy_score(eval_targets,predictions))
print('confusion matrix:\n',confusion_matrix(eval_targets,predictions))
print('classification report:\n',classification_report(eval_targets,predictions))

accuracy: 0.8602
confusion matrix:
 [[1970  504]
 [ 195 2331]]
classification report:
               precision    recall  f1-score   support

         0.0       0.91      0.80      0.85      2474
         1.0       0.82      0.92      0.87      2526

    accuracy                           0.86      5000
   macro avg       0.87      0.86      0.86      5000
weighted avg       0.87      0.86      0.86      5000



## 5. Export Model 

In [171]:
# Saving the files for inference

output_model_file = './models/pytorch_distilbert_imdb.bin'
output_vocab_file = './models/vocab_distilbert_imdb.bin'

torch.save(model, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

print('Saved')

Saved
